# Introduction to LangChain and RAG

Welcome to the first part of our demo! In this session, we'll explore how to build AI-powered applications using **LangChain**, a popular framework for developing applications with Large Language Models (LLMs). We'll start with a simple chatbot and then enhance it with Retrieval Augmented Generation (RAG).

## Setting Up Our Environment

In [1]:
#%pip install --upgrade --quiet  langchain-openai
#%pip install python-dotenv --quiet
#%pip install langchain-chroma --quiet

In [2]:
# import getenv
import os
import dotenv
# Load environment variables from .env file
dotenv.load_dotenv()
# Import Azure OpenAI
from langchain_openai import AzureChatOpenAI

## Building a Simple ChatBot

Let's start with creating a basic chatbot using **LangChain**. We'll use:
- `ChatOpenAI`: The interface to OpenAI's chat models
- `SystemMessage`: Defines the bot's behavior and role
- `HumanMessage`: Represents user input

Our chatbot will act as a Financial Analyst. We'll create it by:
1. Instantiating the model
2. Defining a system prompt that sets the bot's role
3. Sending a user query and getting a response with `.invoke()`

This demonstrates the basic pattern of LLM interactions: prompt → response.

In [3]:
from IPython.display import Markdown
from langchain_core.messages import HumanMessage, SystemMessage
from langchain_openai import AzureChatOpenAI

In [4]:
# Create a ChatOpenAI instance with the LLM model and temperature
base_model = AzureChatOpenAI(
    azure_endpoint=os.getenv("AZURE_OPENAI_ENDPOINT"),
    azure_deployment=os.getenv("AZURE_OPENAI_DEPLOYMENT_NAME"),
    api_key=os.getenv("AZURE_OPENAI_API_KEY"),
    openai_api_version=os.getenv("OPENAI_API_VERSION"),
)

In [5]:
BASE_PROMPT = """
You are a Financial Analyst. Do your best to help the client with their request based on your expertise. Give a succinct and clear response.
"""

In [6]:
# Request from the client
request = "I want to invest in the technology sector. Can you please define an investment strategy?"

# Message list for the base model
messages = [
    SystemMessage(BASE_PROMPT),
    HumanMessage(request),
]

# Invoke the model with the messages
response = base_model.invoke(messages)
Markdown(response.content)

Certainly! Here's a concise investment strategy for the technology sector:

1. **Define Your Goals and Risk Tolerance**: Determine if you’re seeking growth (higher risk) or income (lower risk). Technology tends to be growth-focused but can vary by sub-sector.

2. **Diversify Across Sub-Sectors**: Invest across areas like software (e.g., cloud computing, AI), hardware (e.g., semiconductors), and emerging tech (e.g., blockchain and autonomous systems) to mitigate risk.

3. **Focus on Market Leaders and Innovators**: Consider well-established players (e.g., Apple, Microsoft, NVIDIA) for stability and smaller, innovative firms for higher growth potential.

4. **Assess Financial Health and Valuation Metrics**: Evaluate revenue growth, operating margins, and cash flow. Prioritize companies with financially strong balance sheets and sustainable growth.

5. **Use ETFs for Diversification**: Consider technology-focused ETFs (e.g., XLK, ARKK) as a simple way to gain broad exposure while reducing single-stock risk.

6. **Stay Updated on Trends and Regulations**: The tech sector evolves rapidly; track developments in AI, cybersecurity, IoT, and any potential regulatory impacts.

7. **Set an Exit Plan**: Determine when to take profits or cut losses. Consider rebalancing to avoid overexposure as the sector can be volatile.

8. **Long-Term Perspective**: While tech stocks can be very volatile short-term, historically, they’ve delivered substantial long-term returns.

Remember to review your portfolio periodically and consult a financial advisor to align with your specific financial situation.

## Understanding Retrieval Augmented Generation (RAG)

Now comes the exciting part! RAG is a technique that enhances LLM responses by giving them access to external knowledge. Instead of relying solely on the model's training data, we can provide relevant information from our own database.

### Vector Database Setup

To implement RAG, we need:
1. A collection of documents (in our case, a currated set of 1'000 articles from Bloomberg financial news)
2. A way to search these documents efficiently (vector database)
3. A function to retrieve relevant information based on queries

Here we use:
- `Chroma`: A vector database for storing and retrieving documents
- `OpenAIEmbeddings`: Converts text into vector representations

Let's first set up the global configuration for our retriever.

In [7]:
EMBEDDING_MODEL = "text-embedding-3-small"
RETRIEVAL_K = 5

We'll then define helper functions to load our documents and store them in a vector store.

In [8]:
import pickle

from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_openai import AzureOpenAIEmbeddings

In [9]:
def load_documents(pickle_filepath: str) -> list[Document]:
    """Load documents from a pickle file."""
    with open(pickle_filepath, "rb") as file:
        return pickle.load(file)


def initialize_vector_store(document_chunks: list[Document]) -> Chroma:
    """Reset the Chroma collection and initialize a vector store using document chunks."""
    Chroma().reset_collection()
    embedding_model = AzureOpenAIEmbeddings(model=EMBEDDING_MODEL)
    return Chroma.from_documents(documents=document_chunks, embedding=embedding_model)

Show the docs inside the pickle file

In [10]:
# show docs in the pickle file
with open("./data/bloomberg_financial_news_1k.pkl", "rb") as file:
    loaded_documents = pickle.load(file)

print(loaded_documents)

# print the number of documents
print()
print("Number of documents in the pickle file:")
print(f"Number of documents: {len(loaded_documents)}")

[Document(metadata={'Headline': 'Kuwait Wealth Fund Doubles Investments in U.K. to $24 Billion', 'Link': 'http://www.bloomberg.com/news/2013-06-30/kuwait-wealth-fund-doubles-investments-in-u-k-to-24-billion.html'}, page_content='Kuwait Investment Authority , the country’s sovereign wealth fund, doubled investments in the U.K. over the last 10 years to more than $24 billion. Investment in Britain was “across all asset classes, sectors and industries,” Managing Director Bader Al-Saad said in a speech in London on June 28. Kuwait Investment Office, the KIA’s London-based investment arm, now manages more than $120 billion globally compared with $27 billion 10 years ago. “Alongside the real estate subsidiary, which manages our real estate assets, we have set up an infrastructure fund which we are operating from London,” Al-Saad said. The investment office “will be looking to manage more funds, in more asset classes, and in more regions of the world in the near future,” he said. The KIA, whi

Let's load our documents and inspect the first one.

In [11]:
data_dir = "./data"
data_file = "bloomberg_financial_news_1k.pkl"

# Load the Bloomberg documents from the pickle file
documents = load_documents(os.path.join(data_dir, data_file))

In [12]:
doc_str = f"{documents[0].metadata['Headline']}\n\n{documents[0].page_content}"
Markdown(doc_str)

Kuwait Wealth Fund Doubles Investments in U.K. to $24 Billion

Kuwait Investment Authority , the country’s sovereign wealth fund, doubled investments in the U.K. over the last 10 years to more than $24 billion. Investment in Britain was “across all asset classes, sectors and industries,” Managing Director Bader Al-Saad said in a speech in London on June 28. Kuwait Investment Office, the KIA’s London-based investment arm, now manages more than $120 billion globally compared with $27 billion 10 years ago. “Alongside the real estate subsidiary, which manages our real estate assets, we have set up an infrastructure fund which we are operating from London,” Al-Saad said. The investment office “will be looking to manage more funds, in more asset classes, and in more regions of the world in the near future,” he said. The KIA, which has about $342 billion of assets according to the Sovereign Wealth Fund Institute in California , holds stakes in Daimler AG and BP Plc. (BP/) The fund manages Kuwait’s General Reserve Fund and Future Generations Fund, and invests in local, Arab and international markets.

Show the second one

In [13]:
doc_str = f"{documents[0].metadata['Headline']}\n\n{documents[1].page_content}"
Markdown(doc_str)

Kuwait Wealth Fund Doubles Investments in U.K. to $24 Billion

Sprint Corp. (S) raised $6.5 billion yesterday in the biggest high-yield offering since 2008 as sales of speculative-grade securities surpass last year’s record pace. Sprint’s $4.25 billion of 7.875 percent, 10-year notes priced to yield 498 basis points more than similar-maturity Treasuries and its $2.25 billion of 7.25 percent, eight-year debt pays a spread of 466 basis points, according to data compiled by Bloomberg. A basis point is 0.01 percentage point. Proceeds from the issue by the third-largest U.S wireless carrier will be used for general corporate purposes, Bloomberg data show. The securities are rated B1 by Moody’s Investors Service, four levels below investment grade, and one step higher at BB- at Standard & Poor’s. Sales of dollar-denominated junk-rated debt this year of $242.5 billion exceed the $202.9 billion sold in the corresponding period of 2012, Bloomberg data show. Borrowers issued a record $357.3 billion of high-yield debt last year. The sale is the largest high-yield offering since June 2008, when Intelsat priced $7.1 billion of bonds through three units, Bloomberg data show. Yields on the Bank of America Merrill Lynch U.S. High Yield index have declined to 6.86 percent as of yesterday from a record high 22.65 percent during the credit crisis in December 2008 and reached an unprecedented low of 5.98 percent on May 9. Offering Increased The size of Sprint’s sale was earlier set at a minimum of $4 billion, a person familiar with the transaction said yesterday. JPMorgan Chase & Co., the largest underwriter of junk debt, led the offering. Sprint was bought by SoftBank Corp. in July for $21.6 billion in founder Masayoshi Son’s bid to make the Overland Park, Kansas-based company a stronger rival against Verizon Communications Inc. and AT&T Inc., owners of the two biggest U.S. wireless carriers. Tokyo-based SoftBank’s takeover included a $5 billion cash infusion, giving Sprint money to bolster its network and pursue acquisitions. Verizon is planning to sell as much as $50 billion of investment-grade bonds to fund its purchase of Vodafone Group Plc’s 45 percent stake in Verizon Wireless, a person briefed on the company’s plans said Sept. 2.

### Initializing the Vector Store and the Retriever

The vector store and retriever are key components of our RAG system. Here's what happens in this section:

- Initialize a new Chroma vector store with these documents
- Create a retriever that will fetch the `RETRIEVAL_K` most relevant documents according to their embedding

In [14]:
# Initialize the vector store with the documents
vector_store = initialize_vector_store(documents)

# Create a retriever instance from the vector store
retriever = vector_store.as_retriever(search_kwargs={"k": RETRIEVAL_K})

The retriever acts like a smart search engine - when given a question or topic, it returns the most relevant documents from our database. It does so by finding the documents similar embeddings to the query. In LangChain, this is also done with `.invoke()`. Let's try an example.

In [15]:
# Define the retrieval query
retrieval_query = "What are the latest trends in the technology sector?"
#retrieval_query = "what is the most advanced semiconductor company?"

# Invoke the retriever with the retrieval query
retrieved_documents = retriever.invoke(retrieval_query)

In [16]:
display(retrieved_documents)

[Document(id='c8115140-d68f-4e2f-a8a7-6b31ca397a67', metadata={'Headline': 'ARM Sees Shift to Cheaper Smartphones Boosting Chip Royalty', 'Link': 'http://www.bloomberg.com/news/2013-11-20/arm-sees-shift-toward-cheaper-smartphones-boosting-chip-royalty.html'}, page_content='ARM Holdings Plc (ARM) , the U.K. chip designer whose products power Apple Inc. (AAPL) ’s iPhones, predicts a shift in demand toward lower-priced smartphones will help boost royalty revenue. “There is a shift advantage for ARM as mid-range and entry smartphones drive smartphone growth. We’ve got that dynamic going into next year,” Chief Financial Officer Tim Score said at a conference in Barcelona yesterday. “Those phones generate much more royalty for ARM because they use more silicon.” ARM has had five consecutive years of revenue growth as its designs for chips that control graphics and processing were used in increasingly popular smartphones and tablets. Royalty payments took the spotlight last quarter as they mi

### Creating the RAG System

We can now augment our basic chatbot by providing it access to the retriever using **LangChain** tools, which allow the model to:
- Query the document database if needed
- Provide an answer based on the retrieved documents

#### Creating a tool with LangChain

We can create a tool using the `@tool` decorator from **LangChain** and provide it to the model using `.bind_tools()`. The model will receive all the relevant information about the tool thanks to the decorator. This way it knows how it works and can decide when to use it.

In [17]:
from langchain_core.tools import tool

In [18]:
@tool
def retrieval(retrieval_query: str) -> list[Document]:
    """Retrieve documents based on a query."""
    return retriever.invoke(retrieval_query)


# Create a list of tools and a dictionnary of tool functions by name
tools = [retrieval]
tools_by_name = {tool.name: tool for tool in tools}

In [19]:
RAG_PROMPT = """
You are a Financial Analyst with access to a Bloomberg Financial News database.

Query the database to help the client with their request. Give a succinct and clear response based on the information you find.
"""

# Create the RAG model by binding the base model with the retrieval tool
rag_model = base_model.bind_tools(tools)

Let's check the answer. As we can see its content is empty, but a tool call has been made.

In [20]:
request = "I want to invest in the technology sector. Can you please define an investment strategy?"

# Message list for the RAG model
messages = [
    SystemMessage(RAG_PROMPT),
    HumanMessage(request),
]

# Invoke the RAG model with the messages
rag_response = rag_model.invoke(messages)

Markdown(f"Content: {rag_response.content}\n\nTool Calls: {rag_response.tool_calls}")

Content: 

Tool Calls: [{'name': 'retrieval', 'args': {'retrieval_query': 'latest investment strategies in the technology sector'}, 'id': 'call_hlTfcIB5XLJ8Qz8s8kWcHmPA', 'type': 'tool_call'}]

Let's use the retrieval tool to retrieve documents following the model's query.

In [21]:
# Check if the RAG model response contains tool calls
if rag_response.tool_calls:
    # Get the first tool call from the response
    tool_call = rag_response.tool_calls[0]

    # Get the tool from the tool call
    tool = tools_by_name[tool_call["name"]]

    # Invoke the tool with the tool call arguments
    documents = tool.invoke(tool_call["args"])

    # Combine the retrieved documents into a single string
    documents_str = "\n\n".join(
        [f"{doc.metadata['Headline']}\n\n{doc.page_content}\n" for doc in documents]
    )

In [22]:
Markdown(documents_str)

Mairs & Power’s Frels, Henneman Named Manager of the Year

William Frels and Mark Henneman, who run the $2.5 billion Mairs & Power Growth Fund (MPGFX) , were named domestic stock-fund managers of the year by Morningstar Inc. (MORN)  The top honor for international stock funds went to the $1.3 billion Virtus Foreign Opportunities Fund and the $6.8 billion Virtus Emerging Markets Opportunities Fund, both run by Rajiv Jain, Chicago-based Morningstar said in a statement today. The fixed-income winner was Mark Kiesel , manager of the $10.2 billion Pimco Investment Grade Corporate Bond Fund. The six-member team that runs the $1.8 billion TFS Market Neutral Fund (TFSMX) won in the alternatives category. David Giroux of the $13.7 billion T. Rowe Price Capital Appreciation Fund was named allocation fund manager of the year. Both are new categories for the research firm’s awards. The Mairs & Power Growth Fund returned an average of 5.6 percent annually over the past five years, better than 89 percent of peers, according to data compiled by Bloomberg. Last year it returned 22 percent, topping 95 percent of rivals. Of the fund’s largest 25 holdings , 18 were bought in the 1990s and 17 have been in the portfolio for at least 10 years, Morningstar said in the statement. Virtus Emerging Markets Opportunities Fund (HIEMX) gained 3.8 percent annually for the past five years, better than 94 percent of competitors. The fund returned 20 percent in 2012, besting 58 percent of peers, according to the data. Kiesel’s Pimco fund outperformed 91 percent of similar funds over the past five years, returning 10 percent annually. The fund, which topped 77 percent of rivals in 2012, had 17 percent of its money in bank debt as of Sept. 30. Market Neutral TFS Market Neutral Fund gained 4.5 percent annually for the past five years, better than 97 percent of peers. The team’s 7.8 return last year topped 84 percent of rivals, according to data compiled by Bloomberg. Morningstar defines alternative investments as those that don’t fit neatly into traditional stock or fixed-income categories. T. Rowe Price Capital Appreciation gained 6 percent a year for the past five years, ahead of 95 percent of rivals. Its largest holding was Waltham, Massachusetts-based Thermo Fisher Scientific Inc. (TMO) as of Sept. 30, the data show. Giroux’s 15 percent return in 2012 beat 82 percent of the competition.


Samsung Boosting Venture-Capital Funding to $1.1 Billion

Samsung Electronics Co. (005930) , the world’s largest maker of mobile phones, said it was boosting venture- capital funding by $100 million to help create new businesses for its unit making memory chips and electronics parts. The company unveiled the Catalyst Fund for early stage startups, as well as a strategy and innovation center in Menlo Park , California , at an event near the site on Sand Hill Road. The company and affiliates also run the $1 billion Samsung Ventures America Fund. “We must reach out to global hot spots and global talent,” Young Sohn, president and chief strategy officer of device solutions for Samsung, said at the event. “This is the epicenter of disruptive forces.” Like other companies such as Intel Corp. (INTC) , Google Inc. and Microsoft Corp., Samsung is trying to use its cash to encourage the development of technology that will benefit its business in the future. Traditional venture-capital investors aren’t investing in basic science and technology startups, creating a gap that Samsung intends to help fill, Sohn said. Samsung’s venture arm, now headquartered in Seoul, has more than $1 billion in assets under management and did 20 deals worth $160 million last year, Sohn said. Investments by the new funds will be in the areas of Internet-connected devices, mobile health, computer-human interface, cloud-data centers and mobile privacy, he said. Shares of the Suwon, South Korea-based company fell 0.1 percent to 1,435,000 won in South Korean trading today.


Kony Solutions Weighs Public Offer as Early as This Year

Kony Solutions Inc. , a provider of mobile-application platforms, has talked to banks including Goldman Sachs Group Inc. (GS) and JPMorgan Chase & Co. (JPM) about selling shares in a public offering as early as this year. The company, which has its headquarters in Orlando , Florida , as well as in London , plans to select bankers this year after introducing a service that will provide easier access to its app platforms, Chief Executive Officer Raj Koneru said in an interview yesterday. The share sale will take place this year or in 2014, he said. “We are talking with a lot of bankers,” Koneru said. The company hasn’t decided how much stock to sell or the amount of funds to raise, he said. Kony, which has yet to make a profit, expects to record $80 million to $100 million in revenue this year, about double its 2012 sales, Koneru said. An initial public offering could raise at least $120 million for Kony, valuing the company at more than $700 million, Tom Taulli , an analyst who covers IPOs, said in an interview. “Enterprise is really hot right now,” Taulli said. “Right place at the right time -- that always helps with an IPO. I suspect they’ll do well.” Spending on app-development tools for multiple mobile operating systems will grow at an average annual rate of 39 percent to reach $8.2 billion in 2016, according to Smith’s Point Analytics LLC. Founded in 2007, Kony sells annual subscriptions for its app platform-service, which allows developers to customize products to their own specifications. The company has raised about $35 million from Georgian Partners and Insight Venture Partners and may use proceeds from an IPO to improve its software and for research and development, Koneru said.


Fewer U.S. Shares Available Fuels Bull Market: Chart of the Day

Repurchases are magnifying gains in U.S. stocks and are poised to lift prices further, according to Pavilion Global Markets Ltd. strategists. As the CHART OF THE DAY shows, the Wilshire 5000 Total Market Index has risen more than the market value of all U.S. companies since the current bull market started in March 2009, according to data compiled by Bloomberg. Through last week, the gap was about 13 percentage points. “The difference comes from a reduction in the number of shares,” Pierre Lapointe , head of global strategy and research at the Montreal-based firm, and two colleagues wrote yesterday in a report. “The resulting de-equitization is giving a boost to this stock-market rally.” S&P 500 calculations are based on 2.3 percent fewer shares than in July 2011, when the total reached its high for the bull market, the strategists estimated. A drop in stock outstanding accounted for 25 percent of the past year’s earnings-per-share growth for companies in the index, they added. Apple Inc., the world’s largest company by market value, increased its stock-buyback program last month by $50 billion. That’s enough to pay for about 12 percent of the Cupertino, California-based company’s shares at their closing price yesterday. The program will run through 2015. “De-equitization is a global phenomenon,” Lapointe and his colleagues added. In most developed markets, FTSE stock indexes rose faster than total market value during the past year, according to their report. This was also the case in Brazil, Mexico and South Korea , three emerging markets.


Northern Trust Profit Rises 1.7% as Market Boosts Assets

Northern Trust Corp. (NTRS) , the third- biggest independent U.S. custody bank, said first-quarter profit rose 1.7 percent as stock-market gains boosted the value of assets the company oversees. Net income increased to $164 million, or 67 cents per diluted share, from $161.2 million, or 66 cents a share, a year earlier, the Chicago-based company said today in a statement. Results missed the 72-cent average estimate of 16 analysts surveyed by Bloomberg. “We have a better macro backdrop with higher market levels this quarter, and we saw good organic growth across all three” of the largest publicly listed custody banks, Brian Bedell, a New York-based analyst at ISI Group Inc., said in an interview before results were released. The custody banks, under pressure to improve profitability hurt by record-low interest rates, are benefiting as global equity markets climb for the second straight year. Northern Trust, led by Chairman and Chief Executive Officer Frederick H. Waddell, followed larger rivals Bank of New York Mellon Corp. (BK) and State Street Corp. (STT) with job cuts in 2012 that, combined with technology initiatives, are designed to increase annual pretax income by $250 million by the end of this year. The Standard & Poor’s 500 Index (SPX) of U.S. stocks returned 14 percent in the 12 months ended March 31, and global stocks, as measured by the MSCI ACWI Index, advanced 11 percent, with reinvested dividends. Rate Impact Low rates hurt custody banks by reducing the return they make on lending and on their own investments. The U.S. Federal Reserve has held its benchmark interest rate at zero to 0.25 percent since December 2008 in an attempt to stimulate borrowing and economic growth. Custody banks keep records, track performance and lend securities for institutional investors, including mutual funds, pension funds and hedge funds. Northern Trust also manages investments for individuals and institutions. Results were announced before the start of regular U.S. trading. Northern Trust gained 7.3 percent this year through yesterday, compared with a 15 percent advance by the Standard & Poor’s 20-company index of asset managers and custody banks. BNY Mellon is scheduled to released earnings tomorrow, followed by State Street on April 19. (Northern Trust is scheduled to hold a conference call for investors at 10 a.m. New York time. The call can be accessed at http://www.northerntrust/financialreleases .)


### We can now add the tool's output to the message chain with `ToolMessage`, so the model can answer based on the retrieved documents.

*Note: Here we use the base model instead of the RAG model to limit our agent to one retrieval call. A fully autonomous agent could decide to make subsequent calls to best answer the request*

In [23]:
from langchain_core.messages import ToolMessage

In [24]:
# Message list with the retrieved documents for the base model
messages = [
    # RAG System prompt
    SystemMessage(RAG_PROMPT),
    # Client request
    HumanMessage(request),
    # RAG model response
    rag_response,
    # Retrieval tool response
    ToolMessage(documents_str, tool_call_id=tool_call["id"]),
]

# Invoke the base model with the messages
response = base_model.invoke(messages)

Markdown(response.content)

To invest strategically in the technology sector, here are some approaches based on recent trends and expert recommendations:

### 1. **Diversification Across Tech Sub-sectors**
   Consider allocating investments among emerging areas like Internet-connected devices, mobile health, human-computer interface technologies, cloud-data centers, and mobile privacy. These areas are projected for significant growth, as highlighted by initiatives such as Samsung’s Catalyst Fund focusing on innovation and development.

### 2. **Opportunities in Venture Capital**
   Investing in funds or companies that support early-stage technology startups can be lucrative. Samsung, for example, is increasing its venture-capital funding to $1.1 billion to advance tech startups in vital sectors. Participate in funds with similar growth objectives.

### 3. **Mobile Application Platforms**
   Companies like Kony Solutions, preparing for IPOs, indicate strong growth potential within mobile application development platforms. This area is expected to see a yearly growth rate of 39% through 2016.

### 4. **Target Leaders in De-Equitization**
   Leading tech giants, such as Apple, are repurchasing stocks at scale, boosting earnings per share and share prices. Look into companies exhibiting strong de-equitization patterns for long-term earnings growth.

### 5. **Investment in Established Growth Funds**
   Review technology-heavy mutual funds like Mairs & Power Growth Fund. Such funds consistently outperform peers with a disciplined, long-term approach to tech investments. They could serve as a safer entry into the sector.

### 6. **Explore International Opportunities**
   Emerging markets with high-tech potential, such as South Korea, Mexico, and Brazil, can offer opportunities for broader exposure to technology advancements globally.

### 7. **Risk Mitigation**
   Investing in alternative funds, such as market-neutral funds, provides protection while gaining exposure to technology.

Engage with equity research or portfolio managers to refine your exposure to tech according to your risk appetite and financial goals.

### Different question now

In [25]:
request = "I want to invest in the phone manufacturer area. Can you please define an investment strategy?"

# Message list for the RAG model
messages = [
    SystemMessage(RAG_PROMPT),
    HumanMessage(request),
]

# Invoke the RAG model with the messages
rag_response = rag_model.invoke(messages)

Markdown(f"Content: {rag_response.content}\n\nTool Calls: {rag_response.tool_calls}")

Content: 

Tool Calls: [{'name': 'retrieval', 'args': {'retrieval_query': 'Current market performance, growth trends, and competitive analysis for phone manufacturers like Apple, Samsung, Xiaomi, etc.'}, 'id': 'call_2A7F689AC8u1GueEFBNG64xp', 'type': 'tool_call'}]

In [26]:
# Check if the RAG model response contains tool calls
if rag_response.tool_calls:
    # Get the first tool call from the response
    tool_call = rag_response.tool_calls[0]

    # Get the tool from the tool call
    tool = tools_by_name[tool_call["name"]]

    # Invoke the tool with the tool call arguments
    documents = tool.invoke(tool_call["args"])

    # Combine the retrieved documents into a single string
    documents_str = "\n\n".join(
        [f"{doc.metadata['Headline']}\n\n{doc.page_content}\n" for doc in documents]
    )

In [27]:
Markdown(documents_str)

Apple Sells More Phones Over the Weekend Than BlackBerry Did Last Quarter

Just about the only thing Apple and BlackBerry still have in common is that they're both named after fruit. On Friday, when Apple fanatics were lined up outside stores around the world for the release of the new iPhones, BlackBerry said it would cut 4,500 jobs and write down as much as $960 million in its fiscal second quarter BlackBerry, which released the flagship Z10 touchscreen phone earlier this year, sold 5.9 million smartphones during the last three months. Apple sold more than 9 million over the weekend. The companies' different trajectories became even more vivid today when BlackBerry said it tentatively agreed to a $4.7 billion buyout by a group led by Fairfax Financial Holdings. Meanwhile, Apple's stock rose 5 percent following the sales announcement, giving the California company a market value of almost $446 billion. In the mobile industry, giants fall fast and hard. Device sales for the once-mighty BlackBerry "are cratering," said Jan Dawson, an analyst at research firm Ovum. Compare that with Apple, which said today the debut of the iPhone 5s and 5c set a sales record for the company. It was the first time Apple began selling its new iPhones in the crucial Chinese market on launch day, which no doubt helped to exceed goals. Apple also said quarterly revenue and gross margin will be at the top end of its prior forecast. During Apple's last quarter -- before the new phones came out -- the company took in $18.2 billion in revenue from the iPhone unit, which doesn't include accessories, software or services. BlackBerry expects its entire company to generate $1.6 billion in revenue in the second quarter. You'd have to add up BlackBerry's last seven quarters of revenue to reach what Apple generated from selling smartphones in one quarter. And it wasn't even a particularly great one for Apple.


Apple CEO Cook Says China Will Overtake U.S. as Biggest Market

Apple Inc. (AAPL) Chief Executive Officer Tim Cook said China will overtake the U.S. to become its largest market, as the iPhone-maker adds more stores and considers debuting new products in the country. “I believe it will become our first,” Cook said in an interview with state-owned Xinhua News yesterday. No timeframe was given for the prediction. The Cupertino, California-based company had $5.7 billion of sales in China during the quarter ended September. U.S. revenue was about $14.4 billion, based on figures in an Oct. 25 earnings statement. Cook also met the chairman of China Mobile Ltd. (941) while in the country, as he seeks to boost cooperation with a wireless operator that has 707 million customers and no agreement to sell iPhones. The CEO is making his second visit to the country in less than a year as Apple tries to reverse its shrinking share of the local smartphone market. Apple intends to open “many more” outlets in China over the next several years, Cook said in the Xinhua interview. The company now has 11 stores in China and Hong Kong. Cook also said he would also “love” to introduce new products in China first, according to the report. The iPhone 5 was released in China in December, almost three months after the U.S. introduction. China Growth The company’s growth in the world’s most populous nation has been limited by it only working with smaller carriers and competition from domestic phone-makers. Its share of the nation’s smartphone market slipped to sixth from fourth in the third quarter, according to researcher IDC. Yesterday, Cook met with Xi Guohua, the chairman on China Mobile at the carrier’s headquarters in Beijing. The two discussed “cooperation,” according to a statement from China Mobile, the world’s largest wireless operator by subscribers. It didn’t elaborate. China Mobile, which accounts for 64 percent of the nation’s mobile users, said last month that it needs to reach agreements on benefit-sharing with Apple before it can begin offering iPhones. The operator also has a homegrown third-generation network that isn’t used by other carriers. Apple distributes iPhone with the nation’s second- and third-largest carriers: China Unicom (Hong Kong) Ltd. (762) and China Telecom Corp. (728)  To contact Bloomberg News staff for this story: Edmond Lococo in Beijing at elococo@bloomberg.net 


ARM Sees Shift to Cheaper Smartphones Boosting Chip Royalty

ARM Holdings Plc (ARM) , the U.K. chip designer whose products power Apple Inc. (AAPL) ’s iPhones, predicts a shift in demand toward lower-priced smartphones will help boost royalty revenue. “There is a shift advantage for ARM as mid-range and entry smartphones drive smartphone growth. We’ve got that dynamic going into next year,” Chief Financial Officer Tim Score said at a conference in Barcelona yesterday. “Those phones generate much more royalty for ARM because they use more silicon.” ARM has had five consecutive years of revenue growth as its designs for chips that control graphics and processing were used in increasingly popular smartphones and tablets. Royalty payments took the spotlight last quarter as they missed analysts’ estimates. ARM shares have fallen almost 9 percent since the earnings report on Oct. 22. “As ARM technology becomes more sophisticated, the royalty percentage goes up,” Score said. “On average, it’s going to very gradually go up.” Global smartphone shipments surged 40 percent last quarter, slowing from 53 percent growth for the previous three-month period, according to researcher IDC. The average selling prices of the devices fell by 13 percent to $317 from a year earlier, because of demand for more affordable handsets. In spite of concerns about slowing growth, especially in China, Score said ARM can continue expanding its customer base and outperforming the semiconductor market in the next five years, driven by cheaper handsets, enterprise networking and micro-controllers. ARM shares fell less than 1 percent to 953.50 pence at 9:16 a.m. in London. New Chief Chief Executive Officer Simon Segars, who succeeded Warren East in July, is confronting a shift in the industry’s growth to low-end smartphones and new entrant Intel Corp. (INTC) , which has begun designing its own chips for mobile phones. ARM, based in Cambridge, England, is also seeking to boost its market share in servers as companies from Facebook Inc. (FB) to Google Inc. (GOOG) look for lower-power machines to drive their data centers. It’s “possible to identify such a market and build a chip specifically for that market, whether it is to handle only storage or only video loading, for example. We can make the chip much more efficient,” Score said at the annual Technology, Media and Telecoms Conference organized by Morgan Stanley. “Everyone knows Intel has a very strong ecosystem around servers, we’re not kidding ourselves,” Score said. “For that reason we see it as further out.”


Samsung Falls After JPMorgan Cuts Estimates on Galaxy S4

Samsung Electronics Co. (005930) , the world’s largest smartphone maker, fell the most in more than nine months in Seoul after analysts at JPMorgan Chase & Co. cut profit estimates, citing slowing demand for its flagship Galaxy S4. The shares declined 6.2 percent to 1,427,000 won as of the close of trade, the largest drop since Aug. 27. The stock was the biggest drag on the MSCI Asia Pacific index. Orders for the S4 smartphone, which went on sale last month, are slowing on weak demand in Europe that may impact profit margins, analysts led by JJ Park said in a report dated yesterday, citing supply chain checks. JPMorgan cut its share-price estimate for Samsung by 9.5 percent to 1.9 million won and lowered its 2013 earnings estimates by 9 percent. “Compared to S3, S4 had stronger momentum in the first quarter of launch,” the analysts wrote, affirming their overweight rating on the stock. “But the following quarter’s shipment is expected to be disappointing and its peak-quarter number seems way below our previous estimates.” JPMorgan now expects Suwon, South Korea-based Samsung’s shipments of the S4 to be 60 million this year, compared with a previous estimate of 80 million. Samsung sold 10 million units of the S4 within 27 days, the company said. It also unveiled a smaller, less-powerful version of the phone last week.


Nokia Rises on Report Huawei Considering Acquisition

Nokia Oyj (NOK1V) pared a 12 percent surge in U.S. trading after Chinese telecommunications company Huawei Technologies Co. refuted a report that it was interested in acquiring the struggling Finnish handset maker. Nokia’s American depositary receipts rose 4.6 percent to $3.86 at the close in New York yesterday. Earlier, they had jumped as high as $4.12 after the Financial Times reported that Huawei is “considering these sorts of acquisitions,” according to Richard Yu, head of the company’s consumer business. He said a combination with Nokia would have “some synergies,” according to the FT. Huawei has “no plans” to acquire Nokia, the Chinese company said in an e-mailed statement yesterday. “In the whole history of our company, we have done very few, very small acquisitions,” Scott Sykes, a spokesman for Huawei, said in a phone interview today. One of the first smartphone makers, Espoo, Finland-based Nokia has lost market share as customer preferences have shifted to touchscreen handsets like Apple Inc. (AAPL) ’s iPhone and Samsung Electronics Co. (005930) ’s Galaxy. Huawei, a maker of telecommunications networking gear, has been pushing into the handset market. The company accounted for 4.3 percent of the world’s smartphone shipments in the first quarter, up from 3.4 percent a year earlier, according to data compiled by Bloomberg. Nokia’s share slipped to 2.8 percent last year from 7.8 percent a year earlier. Samsung led the market in the first quarter with 32 percent, followed by Apple with 17 percent. Chris Hollis, a Nokia spokesman, declined to comment.


### We can now add the tool's output to the message chain with `ToolMessage`, so the model can answer based on the retrieved documents.

*Note: Here we use the base model instead of the RAG model to limit our agent to one retrieval call. A fully autonomous agent could decide to make subsequent calls to best answer the request*

In [28]:
# Message list with the retrieved documents for the base model
messages = [
    # RAG System prompt
    SystemMessage(RAG_PROMPT),
    # Client request
    HumanMessage(request),
    # RAG model response
    rag_response,
    # Retrieval tool response
    ToolMessage(documents_str, tool_call_id=tool_call["id"]),
]

# Invoke the base model with the messages
response = base_model.invoke(messages)

Markdown(response.content)

Based on recent trends and market performance in the smartphone industry, here’s an investment strategy:

1. **Focus on Industry Leaders**: Apple's strong product sales, revenue growth, and global expansion (especially in China) make it a leading contender. Its focus on high-margin products and an expanding ecosystem provide stable long-term investment opportunities.

2. **Diversification for Growth in Emerging Markets**: While premium models dominate developed markets, the rising demand for affordable smartphones indicates growth for companies targeting mid-to-low price segments, such as Xiaomi and ARM Holdings.

3. **Caution with Market Volatility**: Samsung's struggles with flagship Galaxy models suggest a potential risk to profitability. Monitor market trends carefully and focus on companies innovating in product features or expanding revenue streams.

4. **Consider Strategic Partnerships**: Huawei’s rise in market share signals potential synergies. However, acquisition rumors and limited historical acquisitions show uncertainty. Exercise caution with investment in such companies unless clear strategic benefits emerge.

5. **Avoid Legacy Brands**: Brands like BlackBerry and Nokia have shown significant declines in market share and profitability. Avoid investments in companies struggling to adapt to modern consumer demands.

Conclusion: For stable returns, prioritize Apple. For higher risk-reward dynamics, consider companies like ARM Holdings or Xiaomi targeting growth markets. Regularly review global smartphone market trends to adapt this strategy.

## Practical Tips

- Watch the temperature setting: Lower values (like 0) are usually better for factual responses
- Pay attention to the number of retrieved documents (`RETRIEVAL_K`): More isn't always better
- The system prompt is crucial: It sets the context and behavior of your bot

## Conclusion

You just learned how to create a chatbot and augment it with a retrieval tool using **LangChain**, this concludes the first part of our workshop!

In the next section, we'll discover **LangGraph** and show how it allows to build sophisticated and flexible LLMs workflows.